In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
from time import sleep
import tqdm
import pandas as pd
import numpy as np

### Get-Soup function
#### Gets the text from the Website

In [3]:
def getSoup(link):
    '''
    No big explanation needed as copied from the lecture, however I kept it on purpose out of the 'DownloaderMidterm' class, because it is in principle 
    reusable for multiple classes. Therefore it is more efficient to keep it outside of the class instead of using it as a class method for each class
    (although of course here I just have one class so it would be the same).
    '''
    sleep(0.1) #to be kind to the website
    r = requests.get(link)
    r.encoding = 'UTF-8'
    return BeautifulSoup(r.text,'lxml') #returns BeautifulSoup object in 'lxml' style

In [4]:
soup=getSoup('http://wiki.lspace.org/mediawiki/List_of_Pratchett_characters')

In [15]:
links=soup.find('span', id='Ankh-Morpork_and_The_Watch').parent.findNext('ul').find_all('li')
characters= ['http://wiki.lspace.org' + li.find('a')['href'] for li in links]

In [38]:
soup2 = getSoup(characters[0])

In [39]:
soup2.find('table').find('tr').find('b')

<b>Mrs Cake</b>

In [22]:
Name = soup2.find('b', text='Name').parent.next_sibling.next_sibling.text.strip()

'Evadne Cake'

In [29]:
Books=soup2.find('a', title='Bibliography').parent.parent.next_sibling.next_sibling.find_all('a')
book_list=[boo.text.strip() for boo in Books]

In [71]:
def df_create(link_list=characters):

    alle_Namen=[]
    alle_Bucher=[]

    for char in characters:
        soup2 = getSoup(char)
        Name = soup2.find('b', text='Name').parent.next_sibling.next_sibling.text.strip() #finds all the Names except for Mr. Slant because his name is not specified in the right position
        alle_Namen.append(Name)
        Books=soup2.find('a', title='Bibliography').parent.parent.next_sibling.next_sibling.find_all('a')
        book_list=[boo.text.strip() for boo in Books]
        alle_Bucher.append(book_list)
        
    df1= pd.DataFrame({'character_name': alle_Namen}) 
    df1.loc[15,'character_name']='Mr. Slant' # Mr. Slant has to be added because his name is not in the correct position on the wiki page, alternatively 
    #'soup2.find('table').find('tr').find('b')' could have been used to scrape the names from the right hand side table header but then I would not have retrieved the full names
    
    df2= pd.DataFrame(alle_Bucher)
    
    df_wide= pd.concat([df1,df2],axis=1) 
    df_long =df_wide.melt(id_vars='character_name', var_name='key', value_name='book').drop(['key'], axis=1).dropna()
    
    df_long= df_long.sort_values('character_name').reset_index(drop=True)
    
    return df_long

df_create()

,character_name,book
0,Carrot Ironfoundersson,Snuff
1,Carrot Ironfoundersson,Thud!
2,Carrot Ironfoundersson,Men at Arms
3,Carrot Ironfoundersson,Jingo
4,Carrot Ironfoundersson,Night Watch
...,...,...
124,Willikins,Jingo
125,Willikins,Men at Arms
126,Willikins,Thud!
127,Willikins,Feet of Clay


In [44]:
df1= pd.DataFrame({'character_name': alle_Namen}) 
df1.loc[15,'character_name']='Mr. Slant' # Mr. Slant has to be added because his name is not in the correct position on the wiki page, alternatively 
#'soup2.find('table').find('tr').find('b')' could have been used to scrape the names from the right hand side table header but then I would not have retrieved the full names

In [58]:
df2= pd.DataFrame(alle_Bucher)

In [62]:
df_wide= pd.concat([df1,df2],axis=1) 
df_long =df_wide.melt(id_vars='character_name', var_name='key', value_name='book').drop(['key'], axis=1).dropna().reset_index(drop=True)

In [70]:
df_long

,character_name,book
120,Carrot Ironfoundersson,Snuff
115,Carrot Ironfoundersson,Thud!
27,Carrot Ironfoundersson,Men at Arms
60,Carrot Ironfoundersson,Jingo
109,Carrot Ironfoundersson,Night Watch
...,...,...
54,Willikins,Jingo
20,Willikins,Men at Arms
96,Willikins,Thud!
38,Willikins,Feet of Clay
